# 03 - Entraînement et évaluation des modèles multi-sorties
Ce notebook reprend et adapte le script `train_multioutput.py` pour entraîner et évaluer :
- Un pipeline scikit-learn avec un `RandomForestRegressor` multi-sorties et une `LogisticRegression` pour la classification du niveau.
- Un modèle Keras multitâche (optionnel) partageant les mêmes couches de base.


## 📦 Import des bibliothèques nécessaires
- `numpy`, `scipy.sparse` : manipulation des features.
- `sklearn.model_selection`, `ensemble`, `multioutput`, `linear_model`, `metrics` : pour le split, l'entraînement et l'évaluation.
- `joblib` : pour la sérialisation des modèles.
- (Optionnel) `tensorflow.keras` pour le modèle multitâche.


In [30]:
import os
import numpy as np
from scipy import sparse
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score
import joblib


## 🗄️ Chargement des données
Nous chargeons les matrices de features et les vecteurs cibles générés précédemment via `prepare_features`.

In [31]:
def load_data():
    dir_feat = os.path.join('data', 'fiverr_features')
    X = sparse.load_npz(os.path.join(dir_feat, 'X_features.npz'))
    y_prix = np.load(os.path.join(dir_feat, 'y_prix.npy'))
    y_evaluation = np.load(os.path.join(dir_feat, 'y_evaluation.npy'))
    y_niveau = np.load(os.path.join(dir_feat, 'y_niveau.npy'))
    # Combiner price et rating pour le multi-output regressor
    y_reg = np.vstack([y_prix, y_evaluation]).T
    # Supprimer les lignes où price ou rating est NaN
    mask = ~np.isnan(y_reg).any(axis=1)
    X = X[mask]
    y_reg = y_reg[mask]
    y_niveau = y_niveau[mask]
    return X, y_reg, y_niveau

# Chargement
X, y_reg, y_niveau = load_data()


## 📈 Split train/test
On sépare 20 % des données pour le test, avec `random_state=42` pour la reproductibilité.

In [32]:
X_train, X_test, y_reg_train, y_reg_test, y_niveau_train, y_niveau_test = train_test_split(
    X, y_reg, y_niveau, test_size=0.2, random_state=42
)
print(f"Train samples: {X_train.shape[0]}, Test samples: {X_test.shape[0]}")


Train samples: 1007, Test samples: 252


## 🤖 Entraînement scikit-learn
- **MultiOutputRegressor** avec un `RandomForestRegressor` (100 arbres).
- **LogisticRegression** pour classifier le niveau.

In [33]:
rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
mor = MultiOutputRegressor(rf)
mor.fit(X_train, y_reg_train)

clf = LogisticRegression(max_iter=500, n_jobs=-1)
clf.fit(X_train, y_niveau_train)


LogisticRegression(max_iter=500, n_jobs=-1)

## 📊 Évaluation des modèles scikit-learn
- MAE, RMSE, R² pour les régressions
- Accuracy pour la classification du niveau

In [34]:
y_reg_pred = mor.predict(X_test)
y_niveau_pred = clf.predict(X_test)
print("Prix MAE:", mean_absolute_error(y_reg_test[:, 0], y_reg_pred[:, 0]))
print("Prix RMSE:", np.sqrt(mean_squared_error(y_reg_test[:, 0], y_reg_pred[:, 0])))
print("Prix R2:", r2_score(y_reg_test[:, 0], y_reg_pred[:, 0]))
print("Note MAE:", mean_absolute_error(y_reg_test[:, 1], y_reg_pred[:, 1]))
print("Note R2:", r2_score(y_reg_test[:, 1], y_reg_pred[:, 1]))
print("Level accuracy:", accuracy_score(y_niveau_test, y_niveau_pred))


Prix MAE: 3.687877619047631
Prix RMSE: 28.741297795221502
Prix R2: 0.6502012328686995
Note MAE: 0.013188888888892667
Note R2: 0.42018711304486656
Level accuracy: 1.0


## 💾 Sauvegarde des modèles
On stocke les modèles scikit-learn pour les réutiliser dans l’application Gradio.

In [35]:
os.makedirs('models', exist_ok=True)
joblib.dump(mor, 'models/mor_rf.pkl')
joblib.dump(clf, 'models/level_clf.pkl')
print("✅ Modèles sauvegardés dans /models")


✅ Modèles sauvegardés dans /models


## 🧠 Entraînement Keras multitâche (optionnel)
Si TensorFlow est installé, on entraîne un MLP multitâche partageant la même base.

In [36]:
try:
    from tensorflow import keras
    from tensorflow.keras import layers
    import tensorflow as tf
except ImportError:
    print("TensorFlow non installé, passage.")
else:
    # Split supplémentaire pour validation
    X_tr, X_val, y_reg_tr, y_reg_val, y_niveau_tr, y_niveau_val = train_test_split(
        X_train, y_reg_train, y_niveau_train, test_size=0.1, random_state=42
    )
    X_tr = X_tr.toarray().astype(np.float32)
    X_val = X_val.toarray().astype(np.float32)
    inp = keras.Input(shape=(X_tr.shape[1],), dtype=tf.float32)
    x = layers.Dense(256, activation='relu')(inp)
    x = layers.Dense(128, activation='relu')(x)
    price_out = layers.Dense(1, name='price')(x)
    rating_out = layers.Dense(1, name='rating')(x)
    level_out = layers.Dense(len(np.unique(y_niveau_tr)), activation='softmax', name='level')(x)
    model = keras.Model(inputs=inp, outputs=[price_out, rating_out, level_out])
    model.compile(
        optimizer='adam',
        loss={'price':'mse','rating':'mse','level':'sparse_categorical_crossentropy'},
        metrics={'price':'mae','rating':'mae','level':'accuracy'}
    )
    model.fit(
        X_tr, [y_reg_tr[:,0], y_reg_tr[:,1], y_niveau_tr],
        validation_data=(X_val, [y_reg_val[:,0], y_reg_val[:,1], y_niveau_val]),
        epochs=10, batch_size=8,
        callbacks=[keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)]
    )
    model.save('models/keras_multi_task.h5')
    print("Modèle Keras multitâche sauvegardé")


Epoch 1/10
114/114 [==============================] - 3s 13ms/step - loss: 5497.0464 - price_loss: 5486.8516 - rating_loss: 9.3134 - level_loss: 0.8819 - price_mae: 14.1754 - rating_mae: 2.1732 - level_accuracy: 0.6336 - val_loss: 2564.7200 - val_price_loss: 2558.6821 - val_rating_loss: 5.2277 - val_level_loss: 0.8104 - val_price_mae: 13.2975 - val_rating_mae: 1.6260 - val_level_accuracy: 0.6535
Epoch 2/10
114/114 [==============================] - 1s 10ms/step - loss: 4723.5801 - price_loss: 4716.5283 - rating_loss: 6.4368 - level_loss: 0.6148 - price_mae: 11.8137 - rating_mae: 1.3457 - level_accuracy: 0.8488 - val_loss: 1969.1049 - val_price_loss: 1965.2747 - val_rating_loss: 3.2563 - val_level_loss: 0.5740 - val_price_mae: 11.6742 - val_rating_mae: 1.2623 - val_level_accuracy: 0.7624
Epoch 3/10
114/114 [==============================] - 1s 9ms/step - loss: 3781.6477 - price_loss: 3778.5232 - rating_loss: 2.5373 - level_loss: 0.5870 - price_mae: 9.0103 - rating_mae: 0.9942 - level_ac